In [ ]:
import tensorflow as tf
import numpy as np

bandits = [] # Список наших бандитов
k = 5 # кол-во бандитов
for i in range(0, k):
    i = np.random.randn(1)
    bandits.append(i)
print (bandits)
num_bandits = len(bandits)
def pullBandit(bandit):
    result = np.random.randn(1) #Сгенерировать случайное число
    if result > bandit:
        return 1 #Выигрыш
    else:
        return -1 #Проигрыш

C:\Users\uivan_000\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\uivan_000\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\uivan_000\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\uivan_000\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Futu

In [20]:
tf.reset_default_graph()
# Эти 2 строчки создают feed-forward часть нейросети. Здесь и происходит выбор действия.
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

# Следующие 6 строчек устанавливают процедуру обучения. Нейросеть принимает на вход действие и 
# его результат, чтобы оценить функцию потерь и обновить веса сети.
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [21]:
total_episodes = 1000 #Количество итераций обучения
total_reward = np.zeros(num_bandits) 
e = 0.1 #Вероятность случайного выбора
init = tf.global_variables_initializer()

In [22]:
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
                
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
    
        reward = pullBandit(bandits[action]) 
        
        #Обновляем веса
        _,resp,ww = sess.run([update,responsible_weight,weights], 
                      feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #Обновляем общий выигрыш каждого бандита
        total_reward[action] += reward
        if i % 50 == 0:
            print("Общий выигрыш сейчас равен " + str(num_bandits) + 
            " бандитов: " + str(total_reward))
        i+=1
print("Агент думает, что бандит №" + str(np.argmax(ww)+1) + " лучший")

Общий выигрыш сейчас равен 5 бандитов: [-1.  0.  0.  0.  0.]
Общий выигрыш сейчас равен 5 бандитов: [-1. -2.  0. 31.  1.]
Общий выигрыш сейчас равен 5 бандитов: [-1. -1.  1. 67.  1.]
Общий выигрыш сейчас равен 5 бандитов: [  0.  -1.   1. 100.   1.]
Общий выигрыш сейчас равен 5 бандитов: [  1.  -1.   0. 138.   1.]
Общий выигрыш сейчас равен 5 бандитов: [  1.  -2.  -1. 173.   0.]
Общий выигрыш сейчас равен 5 бандитов: [  1.  -3.  -2. 213.   0.]
Общий выигрыш сейчас равен 5 бандитов: [  0.  -3.  -2. 249.  -1.]
Общий выигрыш сейчас равен 5 бандитов: [  0.  -2.  -3. 284.   0.]
Общий выигрыш сейчас равен 5 бандитов: [ -1.   0.  -3. 324.   1.]
Общий выигрыш сейчас равен 5 бандитов: [  0.   1.  -3. 354.   1.]
Общий выигрыш сейчас равен 5 бандитов: [  0.   1.  -2. 382.   0.]
Общий выигрыш сейчас равен 5 бандитов: [  2.   1.  -4. 419.  -1.]
Общий выигрыш сейчас равен 5 бандитов: [  2.   2.  -7. 459.  -1.]
Общий выигрыш сейчас равен 5 бандитов: [  1.   2.  -7. 494.  -3.]
Общий выигрыш сейчас раве